# Importing Libraries

In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn import linear_model
import matplotlib.pyplot as plt

In [3]:
#!pip install nfl-data-py
import nfl_data_py as nfl

#!pip install pyarrow
#!pip install pyarrow.parquet
import pyarrow as pa
import pyarrow.parquet as pq

In [4]:
# years_required_1 =[2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]
# NFL_PBP_Data = nfl.import_pbp_data(years = years_required_1, downcast=True, cache=False, alt_path=None)

In [5]:
# print("shape of the data ",NFL_PBP_Data.shape)
# NFL_PBP_Data.head(3)

In [6]:
# NFL_PBP_Data.describe()

## saving the df in an form of table/Parquet file for exporting

In [7]:
# NFL_PBP_Data_table = pa.Table.from_pandas(NFL_PBP_Data)

In [8]:
# pq.write_table(NFL_PBP_Data_table, 'NFL_PBP_Data_2010_2022_original.parquet', compression='GZIP')

In [9]:
# pd.read_parquet('NFL_PBP_Data_2010_2022_original.parquet')

## Download data from S3

In [10]:
#list out buckets
!aws s3 ls

2022-09-28 05:00:55 capstone-nfl-data


In [11]:
#download data
!aws s3 cp s3://capstone-nfl-data/NFL_PBP_Data_2010_2022_original.parquet NFL_PBP_Data_2010_2022_original.parquet

download: s3://capstone-nfl-data/NFL_PBP_Data_2010_2022_original.parquet to ./NFL_PBP_Data_2010_2022_original.parquet


In [12]:
#Load Data
df = pd.read_parquet('NFL_PBP_Data_2010_2022_original.parquet', engine='pyarrow')
df.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,offense_formation,offense_personnel,defenders_in_box,defense_personnel,number_of_pass_rushers,players_on_play,offense_players,defense_players,n_offense,n_defense
0,1.0,2010_01_ARI_STL,2010091208,LA,ARI,REG,1,None,None,None,...,None,None,<NA>,None,<NA>,None,None,None,NaN,NaN
1,36.0,2010_01_ARI_STL,2010091208,LA,ARI,REG,1,ARI,away,LA,...,None,None,<NA>,None,<NA>,None,None,None,NaN,NaN
2,58.0,2010_01_ARI_STL,2010091208,LA,ARI,REG,1,ARI,away,LA,...,None,None,<NA>,None,<NA>,None,None,None,NaN,NaN
3,82.0,2010_01_ARI_STL,2010091208,LA,ARI,REG,1,ARI,away,LA,...,None,None,<NA>,None,<NA>,None,None,None,NaN,NaN
4,103.0,2010_01_ARI_STL,2010091208,LA,ARI,REG,1,ARI,away,LA,...,None,None,<NA>,None,<NA>,None,None,None,NaN,NaN


## Basic Clean Up  (with smaller sample size)

In [13]:
years_required_2 =[2019,2020,2021]
#NFL_PBP_Data_small = nfl.import_pbp_data(years = years_required_2, downcast=True, cache=False, alt_path=None)

NFL_PBP_Data_small = df[ (df.game_date > '2019-06-01') & (df.game_date < '2022-06-01')]

In [15]:
print(NFL_PBP_Data_small.shape)
NFL_PBP_Data_small.head()


(147260, 384)


,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,offense_formation,offense_personnel,defenders_in_box,defense_personnel,number_of_pass_rushers,players_on_play,offense_players,defense_players,n_offense,n_defense
434967,1.0,2019_01_ATL_MIN,2019090804,MIN,ATL,REG,1,None,None,None,...,None,None,<NA>,None,<NA>,,,,0.0,0.0
434968,36.0,2019_01_ATL_MIN,2019090804,MIN,ATL,REG,1,ATL,away,MIN,...,None,None,<NA>,None,<NA>,48000;46263;45215;42450;43971;37179;43449;4235...,00-0035150;00-0033336;00-0032918;00-0032422;00...,00-0034411;00-0031570;00-0028042;00-0030968;00...,11.0,11.0
434969,51.0,2019_01_ATL_MIN,2019090804,MIN,ATL,REG,1,ATL,away,MIN,...,SINGLEBACK,"1 RB, 2 TE, 2 WR",8,"4 DL, 3 LB, 4 DB",7,47814;37179;41235;35485;42354;41232;34472;3553...,00-0035235;00-0028042;00-0031279;00-0026997;00...,00-0031256;00-0027885;00-0031546;00-0027701;00...,11.0,11.0
434970,79.0,2019_01_ATL_MIN,2019090804,MIN,ATL,REG,1,ATL,away,MIN,...,I_FORM,"2 RB, 0 TE, 3 WR",6,"4 DL, 2 LB, 5 DB",<NA>,47814;35485;41235;41808;42354;41232;34472;4334...,00-0035235;00-0030968;00-0031279;00-0026997;00...,00-0027885;00-0031256;00-0031546;00-0032389;00...,11.0,11.0
434971,100.0,2019_01_ATL_MIN,2019090804,MIN,ATL,REG,1,ATL,away,MIN,...,SHOTGUN,"1 RB, 1 TE, 3 WR",6,"4 DL, 2 LB, 5 DB",<NA>,47814;41235;42354;41232;34472;43343;43516;4337...,00-0035235;00-0031279;00-0026997;00-0032392;00...,00-0031256;00-0031546;00-0032389;00-0032448;00...,11.0,11.0


In [11]:
#NFL_PBP_Data_small.to_excel("NFL_PBP_Data_small.xlsx")

In [16]:
# player information for replacing player IDs for individual
nfl.import_depth_charts(years=years_required_2)

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name
0,2019.0,ATL,1.0,REG,1,Bryant,Steven,Matt,Special Teams,00-0020578,3,K,BRY500226,K,Matt Bryant
1,2019.0,ATL,2.0,REG,1,Bryant,Steven,Matt,Special Teams,00-0020578,3,K,BRY500226,K,Matt Bryant
2,2019.0,ATL,3.0,REG,1,Bryant,Steven,Matt,Special Teams,00-0020578,3,K,BRY500226,K,Matt Bryant
3,2019.0,ATL,4.0,REG,1,Bryant,Steven,Matt,Special Teams,00-0020578,3,K,BRY500226,K,Matt Bryant
4,2019.0,ATL,5.0,REG,1,Bryant,Steven,Matt,Special Teams,00-0020578,3,K,BRY500226,K,Matt Bryant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37482,2021.0,HOU,15.0,REG,2,Morrissey,Jimmy,Jimmy,Offense,00-0036859,79,C,MOR678529,C,Jimmy Morrissey
37483,2021.0,HOU,16.0,REG,2,Morrissey,Jimmy,Jimmy,Offense,00-0036859,79,C,MOR678529,C,Jimmy Morrissey
37484,2021.0,HOU,14.0,REG,1,Morrissey,Jimmy,Jimmy,Offense,00-0036859,79,C,MOR678529,C,Jimmy Morrissey
37485,2021.0,HOU,10.0,REG,2,Morrissey,Jimmy,Jimmy,Offense,00-0036859,79,C,MOR678529,C,Jimmy Morrissey


In [17]:
print(NFL_PBP_Data_small.season.unique())
print(NFL_PBP_Data_small.play_type.unique())

[2019 2020 2021]
[None 'kickoff' 'pass' 'run' 'punt' 'no_play' 'extra_point' 'qb_kneel'
 'field_goal' 'qb_spike']


### Truncating the data to just Season 2020 for quicker modeling

In [18]:
nfl_df = NFL_PBP_Data_small.loc[NFL_PBP_Data_small['season'] == 2020 ]
print(nfl_df.season.unique())
print(nfl_df.shape)

[2020]
(48514, 384)


### Removing Kneels, None, Qb Spikes & no_play 

In [19]:
nfl_df = nfl_df[nfl_df['play_type'].isin(['pass', 'run', 'punt', 'extra_point', 'field_goal']) ]
print(nfl_df.shape)

(38641, 384)


### Removing overtime - teams will act very different during overtime...

In [20]:
nfl_df = nfl_df[~(nfl_df['game_half'] == 'Overtime' )]
print(nfl_df.shape)

(38473, 384)


### Normalizing the Yardline as an absolute matrix (Yards to End-Zone)

In [21]:
#  yardline currently moves from 0 to 50.. so '30' could be subjective based on who possesses the ball 
#so I am normalizing it to 0 to 99 to make it easy to follow..

#Using yardline_100 much clearer matrix

nfl_df = nfl_df[nfl_df['yardline_100'].notna()]
print(nfl_df.shape)

(38473, 384)


### Parsing the offense_personnel to pimary positions

In [22]:
print(nfl_df.offense_personnel.unique())

['2 RB, 1 TE, 2 WR' '1 RB, 1 TE, 3 WR' None '1 RB, 2 TE, 2 WR'
 '1 RB, 0 TE, 4 WR' '2 RB, 2 TE, 1 WR' '2 RB, 0 TE, 3 WR'
 '2 RB, 3 TE, 0 WR' '1 RB, 3 TE, 1 WR' '2 QB, 1 RB, 1 TE, 2 WR'
 '1 RB, 1 TE, 2 WR,1 LB' '0 RB, 1 TE, 4 WR' '0 RB, 2 TE, 3 WR'
 '0 RB, 3 TE, 2 WR' '1 RB, 2 TE, 1 WR,1 LB' '6 OL, 1 RB, 1 TE, 2 WR'
 '6 OL, 1 RB, 1 TE, 1 WR,1 LB' '2 RB, 1 TE, 1 WR,1 P,2 LB,1 LS,3 DB'
 '6 OL, 1 RB, 2 TE, 0 WR,1 DL' '6 OL, 1 RB, 2 TE, 1 WR' '0 RB, 0 TE, 5 WR'
 '3 RB, 1 TE, 1 WR' '6 OL, 2 RB, 2 TE, 0 WR' '6 OL, 2 RB, 1 TE, 1 WR'
 '7 OL, 1 RB, 2 TE, 0 WR' '8 OL, 2 RB, 0 TE, 0 WR'
 '6 OL, 1 RB, 3 TE, 0 WR' '6 OL, 2 RB, 0 TE, 2 WR'
 '2 RB, 2 TE, 1 WR,1 P,2 LB,1 LS,1 DL,1 DB' '2 QB, 1 RB, 0 TE, 3 WR'
 '3 RB, 0 TE, 2 WR' '2 QB, 1 RB, 2 TE, 1 WR'
 '2 QB, 6 OL, 2 RB, 1 TE, 0 WR' '2 QB, 2 RB, 2 TE, 0 WR'
 '1 RB, 0 TE, 1 WR,1 P,2 LB,1 LS,2 DL,3 DB' '1 RB, 3 TE, 0 WR,1 DL'
 '3 RB, 0 TE, 1 WR,1 P,3 LB,1 LS,2 DB'
 '2 RB, 1 TE, 0 WR,1 P,4 LB,1 LS,2 DB' '6 OL, 1 RB, 2 TE, 0 WR,1 LB'
 '6 OL, 1 RB, 0 TE, 

In [23]:
def transform_off_personnel(row):

   rb_count = 0
   te_count = 0
   wr_count = 0
   ol_count = 0
   dl_count = 0
   db_count = 0

   if not pd.isna(row['offense_personnel']):
       personnel = row['offense_personnel'].split(', ')
       for p in personnel:
           if p[2:4] == 'RB':
               rb_count = int(p[0])
           elif p[2:4] == 'TE':
                te_count = int(p[0])
           elif p[2:4] == 'WR':
                wr_count = int(p[0])
           elif p[2:4] == 'OL':
                ol_count = int(p[0])
           elif p[2:4] == 'DL':
                dl_count = int(p[0])
           elif p[2:4] == 'DB':
               db_count = int(p[0])

   return pd.Series([rb_count,te_count,wr_count,ol_count,dl_count, db_count])

nfl_df[['off_rb_count','off_te_count','off_wr_count','off_ol_count','off_dl_count', 'off_db_count']] = nfl_df.apply(transform_off_personnel, axis=1)

#### Validation

In [24]:
nfl_df_2 = nfl_df[['offense_personnel','off_rb_count','off_te_count','off_wr_count','off_ol_count','off_dl_count', 'off_db_count']]

In [25]:
nfl_df_2.drop_duplicates()

,offense_personnel,off_rb_count,off_te_count,off_wr_count,off_ol_count,off_dl_count,off_db_count
483003,"2 RB, 1 TE, 2 WR",2,1,2,0,0,0
483007,"1 RB, 1 TE, 3 WR",1,1,3,0,0,0
483008,None,0,0,0,0,0,0
483011,"1 RB, 2 TE, 2 WR",1,2,2,0,0,0
483014,"1 RB, 0 TE, 4 WR",1,0,4,0,0,0
...,...,...,...,...,...,...,...
523662,"1 RB, 0 TE, 0 WR,1 P,4 LB,1 LS,1 DL,3 DB",1,0,0,0,0,0
525134,"0 RB, 1 TE, 0 WR,1 P,4 LB,1 LS,4 DB",0,1,0,0,0,0
528167,"2 QB, 0 RB, 3 TE, 1 WR",0,3,1,0,0,0
528210,"6 OL, 0 RB, 2 TE, 2 WR",0,2,2,6,0,0


### Parsing the defense_personnel to pimary positions

In [26]:
print(nfl_df.defense_personnel.unique())

['2 DL, 5 LB, 4 DB' '1 DL, 5 LB, 5 DB' None '4 DL, 2 LB, 5 DB'
 '4 DL, 3 LB, 4 DB' '3 DL, 4 LB, 4 DB' '2 DL, 4 LB, 5 DB'
 '4 DL, 1 LB, 6 DB' '3 DL, 5 LB, 3 DB' '4 DL, 5 LB, 2 DB'
 '3 DL, 3 LB, 5 DB' '3 DL, 2 LB, 6 DB' '3 DL, 1 LB, 7 DB'
 '2 DL, 3 LB, 6 DB' '1 DL, 4 LB, 6 DB' '0 DL, 6 LB, 4 DB, 1 WR'
 '6 DL, 3 LB, 2 DB' '5 DL, 2 LB, 4 DB' '5 DL, 1 LB, 5 DB'
 '5 DL, 3 LB, 3 DB' '4 DL, 4 LB, 3 DB' '2 DL, 2 LB, 7 DB'
 '5 DL, 4 LB, 2 DB' '6 DL, 2 LB, 3 DB' '6 DL, 4 LB, 1 DB'
 '1 DL, 3 LB, 7 DB' '0 DL, 2 LB, 5 DB, 2 RB,2 WR' '0 DL, 3 LB, 8 DB'
 '3 DL, 2 LB, 4 DB, 1 RB,1 WR' '2 DL, 4 LB, 4 DB, 1 WR' '5 DL, 1 LB, 4 DB'
 '0 DL, 3 LB, 5 DB, 2 RB,1 WR' '4 DL, 1 LB, 5 DB, 1 TE' '3 DL, 2 LB, 5 DB'
 '5 DL, 0 LB, 6 DB' '0 DL, 5 LB, 6 DB' '4 DL, 0 LB, 7 DB'
 '4 DL, 3 LB, 3 DB, 1 WR' '4 DL, 2 LB, 4 DB, 1 WR' '2 DL, 3 LB, 5 DB'
 '4 DL, 5 LB, 1 DB' '5 DL, 2 LB, 3 DB, 1 WR' '3 DL, 4 LB, 5 DB'
 '0 DL, 6 LB, 5 DB' '5 DL, 5 LB, 1 DB' '4 DL, 6 LB, 1 DB'
 '2 DL, 1 LB, 7 DB, 1 TE' '2 DL, 2 LB, 6 DB, 1 TE' '4 DL

In [27]:
def transform_def_personnel(row):

   dl_count = 0
   db_count = 0
   lb_count = 0
   rb_count = 0
   wr_count = 0
   ol_count = 0
   
   if not pd.isna(row['defense_personnel']):
       personnel = row['defense_personnel'].split(', ')
       for p in personnel:
           if p[2:4] == 'LB':
                lb_count = int(p[0])
           elif p[2:4] == 'DL':
                dl_count = int(p[0])
           elif p[2:4] == 'DB':
                db_count = int(p[0])
           elif p[2:4] == 'WR':
                wr_count = int(p[0])
           elif p[2:4] == 'RB':
                rb_count = int(p[0])
           elif p[2:4] == 'OL':
                ol_count = int(p[0])
                
   return pd.Series([dl_count,db_count,lb_count,rb_count,wr_count,ol_count])

nfl_df[['def_dl_count','def_db_count','def_lb_count','def_rb_count','def_wr_count','def_ol_count']] = nfl_df.apply(transform_def_personnel, axis=1)

#### Validation

In [28]:
nfl_df_2 = nfl_df[[
    'defense_personnel',
                   'def_dl_count',
                   'def_db_count',
                   'def_lb_count',
                   'def_rb_count',
                   'def_wr_count',
                   'def_ol_count']]

In [29]:
nfl_df_2.drop_duplicates()

,defense_personnel,def_dl_count,def_db_count,def_lb_count,def_rb_count,def_wr_count,def_ol_count
483003,"2 DL, 5 LB, 4 DB",2,4,5,0,0,0
483007,"1 DL, 5 LB, 5 DB",1,5,5,0,0,0
483008,None,0,0,0,0,0,0
483010,"4 DL, 2 LB, 5 DB",4,5,2,0,0,0
483011,"4 DL, 3 LB, 4 DB",4,4,3,0,0,0
...,...,...,...,...,...,...,...
522450,"4 DL, 3 LB, 3 DB",4,3,3,0,0,0
523662,"0 DL, 3 LB, 5 DB, 3 WR",0,5,3,0,3,0
527513,"7 DL, 1 LB, 3 DB",7,3,1,0,0,0
528080,"4 DL, 2 LB, 4 DB, 1 OL",4,4,2,0,0,1


### Determining equating pass or rush as numberical

In [30]:
cross_tab = pd.crosstab(index=nfl_df['play_type'],
                        columns=nfl_df['play_type_nfl'])
cross_tab

play_type_nfl,FIELD_GOAL,PASS,PAT2,PUNT,RUSH,SACK,XP_KICK
play_type,,,,,,,
extra_point,0,0,0,0,0,0,1395
field_goal,1007,0,0,0,0,0,0
pass,0,18819,101,0,0,1177,0
punt,0,0,0,1981,0,0,0
run,0,0,39,0,13954,0,0


In [31]:
def PlayType_normalized(s):
  if s['play_type_nfl'] == 'PASS': 
    return 'PASS'
  elif s['play_type_nfl'] == 'RUSH': 
    return 'RUSH'
  else:
    return 'OTHER'

In [32]:
nfl_df['PlayType_normalized'] = nfl_df.apply(PlayType_normalized, axis=1)

In [33]:
#only keeping pass or rushes
nfl_df = nfl_df[~(nfl_df['PlayType_normalized'] == 'OTHER' )]

In [34]:
cross_tab = pd.crosstab(index=nfl_df['PlayType_normalized'],
                        columns=nfl_df['play_type'])
cross_tab

play_type,pass,run
PlayType_normalized,,
PASS,18819,0
RUSH,0,13954


In [35]:
#assing passing as 1 & rush as 0
nfl_df['PlayType_normalized_binary'] = nfl_df['PlayType_normalized'].map(lambda p: 1 if p == 'PASS' else 0)

### binning the distance to end-zone

In [36]:
bins = [0,10,20,30,40,50,60,70,80,90,100]
nfl_df['yardline_binned'] = pd.cut(nfl_df['yardline_100'], bins)

In [37]:
nfl_df[['yardline_binned','yardline_100']]

,yardline_binned,yardline_100
483003,"(70, 80]",75.0
483004,"(50, 60]",55.0
483005,"(40, 50]",41.0
483006,"(30, 40]",39.0
483007,"(40, 50]",45.0
...,...,...
531505,"(20, 30]",26.0
531506,"(20, 30]",26.0
531507,"(20, 30]",25.0
531509,"(10, 20]",14.0


### Subsetting the columns for advance models

In [38]:
nfl_final = nfl_df[['play_id','game_id','home_team','away_team','PlayType_normalized','PlayType_normalized_binary','season_type','posteam','down','ydstogo','yardline_100','yardline_binned',
                   'offense_personnel','off_rb_count','off_te_count','off_wr_count','off_ol_count','off_dl_count','off_db_count',
                   'defense_personnel','def_dl_count','def_db_count','def_lb_count','def_rb_count','def_wr_count','def_ol_count',
                    'offense_formation','defenders_in_box'
                   ]]

## Base Model - Linear Regresssion

In [39]:
# importing train_test_split from sklearn
from sklearn.model_selection import train_test_split
training_data, testing_data = train_test_split(nfl_final, test_size=0.2, random_state=210)
print(f"No. of training examples: {training_data.shape[0]}")
print(f"No. of testing examples: {testing_data.shape[0]}")

No. of training examples: 26218
No. of testing examples: 6555


In [40]:
training_data

,play_id,game_id,home_team,away_team,PlayType_normalized,PlayType_normalized_binary,season_type,posteam,down,ydstogo,...,off_db_count,defense_personnel,def_dl_count,def_db_count,def_lb_count,def_rb_count,def_wr_count,def_ol_count,offense_formation,defenders_in_box
484754,3002.0,2020_01_MIA_NE,NE,MIA,RUSH,0,REG,NE,1.0,10.0,...,0,"4 DL, 3 LB, 4 DB",4,4,3,0,0,0,I_FORM,7
498388,2778.0,2020_06_DET_JAX,JAX,DET,PASS,1,REG,JAX,1.0,5.0,...,0,"4 DL, 2 LB, 5 DB",4,5,2,0,0,0,SHOTGUN,6
487095,1522.0,2020_02_DET_GB,GB,DET,PASS,1,REG,DET,3.0,25.0,...,0,"1 DL, 4 LB, 6 DB",1,6,4,0,0,0,SHOTGUN,4
512490,1465.0,2020_12_CAR_MIN,MIN,CAR,PASS,1,REG,CAR,1.0,10.0,...,0,"4 DL, 2 LB, 5 DB",4,5,2,0,0,0,SHOTGUN,6
508248,2611.0,2020_10_JAX_GB,GB,JAX,RUSH,0,REG,GB,2.0,5.0,...,0,"4 DL, 2 LB, 5 DB",4,5,2,0,0,0,SHOTGUN,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485471,3439.0,2020_01_SEA_ATL,ATL,SEA,RUSH,0,REG,SEA,1.0,10.0,...,0,"4 DL, 2 LB, 5 DB",4,5,2,0,0,0,SHOTGUN,6
486773,3074.0,2020_02_CIN_CLE,CLE,CIN,RUSH,0,REG,CIN,1.0,10.0,...,0,"4 DL, 2 LB, 5 DB",4,5,2,0,0,0,SINGLEBACK,6
511206,588.0,2020_11_NE_HOU,HOU,NE,PASS,1,REG,HOU,3.0,10.0,...,0,"3 DL, 1 LB, 7 DB",3,7,1,0,0,0,SHOTGUN,6
493536,2292.0,2020_04_MIN_HOU,HOU,MIN,RUSH,0,REG,MIN,1.0,10.0,...,0,"3 DL, 4 LB, 4 DB",3,4,4,0,0,0,PISTOL,7


In [41]:
# very simple model of playstyle ~ yardline_100 + down
x_train = training_data[['yardline_100', 'down']]
y_train = training_data[['PlayType_normalized_binary']]
x_test =  testing_data[['yardline_100', 'down']]
y_test =  testing_data[['PlayType_normalized_binary']]

In [42]:
x_train.head(3)

,yardline_100,down
484754,26.0,1.0
498388,8.0,1.0
487095,65.0,3.0


In [43]:
y_train.head(3)

,PlayType_normalized_binary
484754,0
498388,1
487095,1


In [44]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()

In [45]:
regressor.fit(x_train, y_train)

LinearRegression()

In [46]:
y_prediction=regressor.predict(x_test)

In [47]:
import sklearn.metrics as metrics

rounded_predictions = np.round(y_prediction)

accuracy = metrics.accuracy_score(y_test, rounded_predictions)
print("Linear Regression is determining when to pass or rush as a function of Down & yards to end-zone")
print("The accuracy of the base model is","{:.2f}".format(accuracy *100),"%")

Linear Regression is determining when to pass or rush as a function of Down & yards to end-zone
The accuracy of the base model is 58.20 %


# Saving the Model to Hugging Face

In [48]:
! pip install huggingface_hub

     |████████████████████████████████| 120 kB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 2.1 MB/s  eta 0:00:01
     |████████████████████████████████| 661 kB 42.3 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 16.1 MB/s eta 0:00:01
     |████████████████████████████████| 140 kB 116.5 MB/s eta 0:00:01
     |████████████████████████████████| 161 kB 56.9 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 243 kB/s  eta 0:00:01


In [49]:
from huggingface_hub import notebook_login
notebook_login()

In [50]:
from huggingface_hub import HfApi
api = HfApi()
api.create_repo(repo_id="baseline-model", private=True)

'https://huggingface.co/mchawla33/baseline-model'

In [51]:
from joblib import dump, load
dump(regressor, 'regressor.joblib') 

['regressor.joblib']

In [52]:
api.upload_file(path_or_fileobj="regressor.joblib", 
                path_in_repo="regressor.joblib", 
                repo_id="mchawla33/baseline-model",
)

'https://huggingface.co/mchawla33/baseline-model/blob/main/regressor.joblib'

# Download Model

In [56]:
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id="mchawla33/baseline-model", filename="regressor.joblib")

Downloading:   0%|          | 0.00/907 [00:00<?, ?B/s]

'/home/studio-lab-user/.cache/huggingface/hub/models--mchawla33--baseline-model/snapshots/d4b4cb6a5fc57c6a8270e0542adc0d04d9b0e533/regressor.joblib'

In [57]:
download_regressor = load('regressor.joblib') 

In [58]:
y_prediction_D=download_regressor.predict(x_test)

In [59]:
rounded_predictions_D = np.round(y_prediction_D)

accuracy_D = metrics.accuracy_score(y_test, rounded_predictions_D)
print("Linear Regression is determining when to pass or rush as a function of Down & yards to end-zone")
print("The accuracy of the base model is","{:.2f}".format(accuracy_D *100),"%")

Linear Regression is determining when to pass or rush as a function of Down & yards to end-zone
The accuracy of the base model is 58.20 %
